# Data Modeling with Cassandra

## Project Background
<p>A startup called Sparkify wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. The analysis team is particularly interested in understanding what songs users are listening to. Currently, there is no easy way to query the data to generate the results, since the data reside in a directory of CSV files on user activity on the app.</p>

<p>They'd like a data engineer to create an Apache Cassandra database which can create queries on song play data to answer the questions, and wish to bring you on the project. Your role is to create a database for this analysis. You'll be able to test your database by running queries given to you by the analytics team from Sparkify to create the results.</p>

# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        # skipping the header
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
# dialect is used to define the parameters for a specific reader or writer
# quote_all instructs writer objects to quote all fields
# skipinitialspace: when true, whitespace immediately following the delimiter is ignored
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion

#### Creating a Cluster

In [5]:
# make a connection to a Cassandra  
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkifydb
WITH REPLICATION = 
{'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkifydb')

## Create queries to ask the following three questions of the data

## Task 1: Getting the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
<p>For this query sessionId and itemInSession are considered as the composite key, because they are used to part the table. The query is to extract the song record in a specific session with a specific order in that session. Treating sessionId and itemInSession as partition keys means storing data in nodes according to sessionId and itemInSession. When the query is called, the system only needs to go to a certain node with the target data, which makes the fast-read easy.</p> 

In [8]:
# create table for the first query
query1 = "CREATE TABLE IF NOT EXISTS song_record"
query1 = query1 + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"
session.execute(query1)                  

In [9]:
# insert data into the first table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_record (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
# Add in the SELECT statement to verify the data was entered into the table
test1 = "select artist, song, length from song_record WHERE sessionId = 338 and itemInSession = 4"
rows = session.execute(test1)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Task 2: Getting name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
<p>For this query userId, sessionId and itemInSession are considered as the composite key. UserId and sessionId are regarded as the partition keys as they are used to filter data. ItemInSession is considered as clustering key as it is used to sort the result.</p> 

In [11]:
# create table for the second query
query2 = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query2 = query2 + "(userId int, sessionId int,  itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query2)     

In [12]:
# insert data into the second table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# Add in the SELECT statement to verify the data was entered into the table
test2 = "select artist, song, firstname, lastname from song_playlist_session WHERE userId = 10 and sessionId = 182"
rows = session.execute(test2)
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Task 3: Getting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
<p>For this query song and userId are considered as the composite key. As the query is to filter the data according to song title, song is chosen to be the partition key. However, song column alone cannot make each row of record unique as many users might listen to the same song or one user might lisen to the same songs in many sessions. If there are more than one user listening to the song, we can only get one record. So adding userId column to the primary key makes sure that the result will have every user who listened to the target song.</p> 

In [14]:
# create table for the third query
query3 = "CREATE TABLE IF NOT EXISTS userlist_song"
query3 = query3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
session.execute(query3)  

In [16]:
# insert data into the third table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO userlist_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
# Add in the SELECT statement to verify the data was entered into the table
test3 = "select firstname, lastname from userlist_song WHERE song = 'All Hands Against His Own'"
rows = session.execute(test3)
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
# Drop the tables before closing out the sessions

In [18]:
query = "drop table song_record"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table userlist_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()